## Google Trends related queries 
Objective: get the related queries for all date where the peak of interest around coronavirus were reached (for every country). 

Peak of interest is reached when it tends to 100.

In [ ]:
import pandas as pd
from pytrends.request import TrendReq

In [ ]:
interest=pd.read_csv('data/interest_over_time.csv')
interest.sample(10)

In [ ]:
set(interest.Country.values)

In [ ]:
peak_filtered=interest[interest.coronavirus==100]
peak_filtered

In [ ]:
pytrends = TrendReq()

In [ ]:
pytrends.build_payload(['coronavirus'], cat=0, timeframe='2020-03-22 2020-03-22', geo='AU', gprop='')
related_queries=pytrends.related_queries()
rising_queries=related_queries['coronavirus']['rising']

In [ ]:
rising_queries

In [ ]:
rising_dict=rising_queries['query'].head(5).to_dict()
rising_dict

In [ ]:
def get_related_queries(df):
    #print("date",date)
    #print("country:",country)
    country_geo={'Australia':'AU','Canada':'CA','Ireland':'IE','South_Africa':'ZA',
                 'United_Kingdom':'UK','United_States_of_America':'US'}
    
    pytrends.build_payload(['coronavirus'], cat=0, timeframe=f'{df.date} {df.date}', geo=country_geo[df.Country], gprop='')
    
    # Using the related query method of pytrends based on payload information
    related_queries=pytrends.related_queries()
    rising_queries=related_queries['coronavirus']['rising']
    
    # transform the top 5 query into a dictionnary to be put into a new cell
    rising_dict=rising_queries['query'].head(5).to_dict()
    
    return rising_dict

# Testing the function
#get_related_queries(peak_filtered.iloc[5].date,peak_filtered.iloc[5].Country)


In [ ]:
peak_filtered.apply(lambda x: get_related_queries(x),axis=1)

In [ ]:
#peak_filtered['related_queries']=peak_filtered.apply(get_related_queries)